In [18]:
__author__ = "me"
__date__ = "2015_10_13"

%pylab inline
import pandas as pd
import numpy as np
import random

import pylab as plt
import os

from fuzzywuzzy import process

import requests
s = requests.get("https://raw.githubusercontent.com/Casyfill/CUSP_templates/master/Py/fbMatplotlibrc.json").json()
plt.rcParams.update(s)

from geopandas.tools import sjoin
from shapely.geometry import Point

numpy.random.seed(2015)

PARQA = os.getenv('PARQA')

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## PropID

In [2]:
prop = pd.read_excel(PARQA + 'data/Input/Parks_Data/CUSP_Adjusted_Spatial_Data.xlsx')[['ParkID','Name']]
prop = prop.dropna()

In [3]:
def trySplit(x, spl='-'):
    '''get rid of addons'''
    if spl in x:
        return x.split(spl)[0].strip()
    else:
        return x

In [4]:
prop.ParkID = prop.ParkID.apply(trySplit)
prop.Name = prop.Name.apply(lambda x: trySplit(x, spl='Zone'))
# prop['boro'] = prop.ParkID.str[0]

In [5]:
prop = prop.drop_duplicates()
prop.Name = prop.Name.str.lower()
prop['type'] = 'origin'

In [6]:
# prop[(prop.ParkID.str.len() > 4)].sort_values('ParkID')


In [7]:
d = [
    {'ParkID':'B073','type':'abstr','Name': 'prospect park'},
    {'ParkID':'M010','type':'abstr','Name': 'central park'},
    {'ParkID':'Q004','type':'abstr','Name': 'astoria park'},
    {'ParkID':'X010','type':'abstr','Name': 'crotona parks'},
    {'ParkID':'Q162','type':'abstr', 'Name': 'rockaway beach boardwalk'},
    {'ParkID':'M014','type':'abstr', 'Name': 'jackie robinson park'},
    {'ParkID':'M028','type':'abstr', 'Name': 'fort washington park'},
    {'ParkID':'M037','type':'abstr', 'Name': 'highbridge park'},
    {'ParkID':'M098','type':'abstr', 'Name': 'washington square park'},
    {'ParkID':'M105','type':'abstr', 'Name': 'sara d. roosevelt park'},
    {'ParkID':'M107','type':'abstr', 'Name': "hell's kitchen park"},
    {'ParkID':'M283','type':'abstr', 'Name': 'battery park city'},
    {'ParkID':'Q001','type':'abstr', 'Name': 'alley pond park'},
    {'ParkID':'Q005','type':'abstr', 'Name': 'baisley pond park'},
    {'ParkID':'Q009','type':'abstr', 'Name': 'macneil park'},
    {'ParkID':'Q012','type':'abstr', 'Name': 'crocheron park'},
    {'ParkID':'Q020','type':'abstr', 'Name': 'highland park'},
    {'ParkID':'Q021','type':'abstr', 'Name': 'cunningham park'},
    {'ParkID':'Q024','type':'abstr', 'Name': 'kissena park'},
    {'ParkID':'Q102','type':'abstr', 'Name': 'juniper valley park'},
    {'ParkID':'R129','type':'abstr', 'Name': 'greenbelt native plant center'}
    
]

In [8]:
prop = prop.append(pd.DataFrame(d), ignore_index=1)

In [9]:
# prop.tail(25)

In [10]:
# for name, g in prop.groupby('ParkID'):
#     if len(g)>2:
#         print g
#         print

## Matching name to property

In [11]:
prop[prop.Name.str.contains('brownsville')]

,Name,ParkID,type
938,brownsville playground,B270,origin


In [12]:
names = prop.Name.tolist()

In [13]:
# prop['NAME'] = prop.Name

In [14]:
# d = [
#     {'ParkID':'B'},
# ]
# prop.NAME[(prop.ParkID.str.contains('B073'))] = 'prospect park'
# prop.NAME[(prop.ParkID.str.contains('M010'))] = 'central park'
# prop.NAME[(prop.ParkID.str.contains('Q004'))] = 'astoria park'
# prop.NAME[(prop.ParkID.str.contains('X010')) & (prop.NAME.str.contains('crotona'))] = 'crotona parks'
# prop.NAME[(prop.ParkID.isin(['Q164','Q163','Q162','Q050']))] = 'rockaway beach boardwalk'
# prop.NAME[(prop.ParkID.isin(['Q300','Q300A']))] = 'kissena corridor park'
# prop.NAME[(prop.ParkID.isin(['M014']))] = 'jackie robinson park'
# prop.NAME[(prop.ParkID.isin(['M028']))] = 'fort washington park'
# prop.NAME[(prop.ParkID.isin(['M037']))] = 'highbridge park'
# prop.NAME[(prop.ParkID.isin(['M098']))] = 'washington square park'
# prop.NAME[(prop.ParkID.isin(['M105']))] = 'sara d. roosevelt park'
# prop.NAME[(prop.ParkID.isin(['M107']))] = "hell's kitchen park"
# prop.NAME[(prop.ParkID.isin(['M283']))] = "battery park city"
# prop.NAME[(prop.ParkID.isin(['Q001']))] = "alley pond park"
# prop.NAME[(prop.ParkID.isin(['Q005']))] = "baisley pond park"
# prop.NAME[(prop.ParkID.isin(['Q009']))] = "macneil park"
# prop.NAME[(prop.ParkID.isin(['Q012']))] = "crocheron park"
# prop.NAME[(prop.ParkID.isin(['Q020']))] = "highland park"
# prop.NAME[(prop.ParkID.isin(['Q021']))] = "cunningham park"
# prop.NAME[(prop.ParkID.isin(['Q024']))] = "kissena park"
# prop.NAME[(prop.ParkID.isin(['Q102']))] = "juniper valley park"


# kissena corridor park
# kissena park golf course

In [15]:
# DPRprop = prop[['Name','ParkID','boro']].groupby('Name').agg({'Name': lambda x:list(set(x.tolist())), 'ParkID':lambda x: list(set(x.tolist())),'boro':'first'}).reset_index()

In [16]:
# DPRprop['Name'][DPRprop.NAME=='rockaway beach'].tolist()

In [17]:
# prop.to_excel(PARQA + 'data/Input/Parks_Data/prop.xlsx')

## 311 Property Names

In [7]:
onto = pd.read_csv(PARQA + 'parqa/311/ONTOLOGY/onto_data/Ontology_matched.csv', index_col=0)
onto['boro'] = onto.parkDistrict.apply(trySplit)
# onto.head(3)

In [8]:
onto.head(2)

,cleanName,NAME,Type,valid,parkDistrict,lat,lon,boro
0,geo soilan park - battery park city,battery park city,other,av,M-01,-74.016893,40.711882,M
1,geo soilan park - battery park city,battery park city,other,av,M-01,-74.016890,40.712719,M


In [165]:
onto['NAME'][ onto['cleanName'].str.contains('red hook park') ]= 'red hook park'

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [166]:
def getBest(x):
    return process.extractOne(x, names)

In [167]:
onto['bestMatch'] = onto.NAME.apply(getBest)

In [169]:
onto['conf'] = onto.bestMatch.str.get(1)
onto.bestMatch = onto.bestMatch.str.get(0)
onto.conf[onto.bestMatch=='playground'] = 50 ## this match not gonna help us

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [170]:
onto.conf.describe()

count    1793.000000
mean       95.535973
std        10.778615
min        50.000000
25%        96.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: conf, dtype: float64

In [143]:
onto[onto.NAME.str.contains('red hook park')]

,cleanName,NAME,Type,valid,parkDistrict,lat,lon,boro,bestMatch,conf


In [171]:
onto[(onto.conf>80) & (onto.conf<90) ].NAME.value_counts()

miracle garden                                  4
orchard beach and promenade                     3
6bc botanical garden                            3
haffen pool                                     2
recreation center 54                            2
gertrude ederle recreation center               2
st. nicholas - olive st. garden                 2
metropolitan pool                               2
south shore golf course                         2
la tourette golf course                         2
roy wilkins recreation center                   2
lyons pool                                      2
kosciuszko pool                                 2
coney island and coney island boardwalk         2
tony dapolito pool                              2
a.r.r.o.w. field house                          2
clearview golf course                           2
john jay pool                                   2
p.s. 4                                          1
firemen's memorial garden                       1


#### For now: Work only with those with confidence score >90

In [174]:
100.0*len(onto[onto.conf>90])/len(onto)

79.19687674288902

In [172]:
onto[( onto.conf < 90) ]

,cleanName,NAME,Type,valid,parkDistrict,lat,lon,boro,bestMatch,conf
12,col david marcus playground,col david marcus mem playground,pgs,av,B-12,-73.969600,40.609700,B,playground,50
16,mccarren park - vincent v abate playground,vincent v abate playground,pgs,av,B-01,-73.949800,40.721300,B,playground,50
48,may matthews playground,may matthews playground,pgs,av,M-04,-73.992700,40.761400,M,playground,50
54,terrapin playground - jhs 51,terrapin playground,pgs,av,B-06,-73.985500,40.673000,B,playground,50
60,kolbert park,kolbert playground,other,v,B-14,-73.958008,40.620076,B,playground,50
110,recreation center - hansborough,hansborough recreation center,other,av,M-10,-73.939319,40.812828,M,olmsted center,86
136,pool - ps 186 playground,ps 186 playground pool,pool,v,Q-13,-73.720600,40.749800,Q,playground,50
143,recreation center - red hook,red hook recreation area,other,av,B-06,-74.005700,40.671869,B,greenbelt recreation center playground,86
147,pool - metropolitan avenue,metropolitan pool,pool,v,B-01,-73.960300,40.715000,B,the metropolitan museum of art,86
150,recreation center - chelsea,chelsea recreation center,other,av,M-04,-74.002219,40.748298,M,greenbelt recreation center playground,86


In [34]:
ontoConf = onto[onto.conf>90]

## Group DPR Property by park name (for randomization)

In [43]:
DPR = ontoConf.merge(DPRprop, left_on='bestMatch',right_on='NAME', how='left')

In [45]:
DPR.head()

,cleanName,NAME_x,Type,valid,parkDistrict,lat,lon,boro_x,bestMatch,conf,NAME_y,boro_y,ParkID,Name
0,geo soilan park - battery park city,battery park city,other,av,M-01,-74.016893,40.711882,M,battery park city,100,battery park city,M,[M283],"[nelson rockefeller park playground north, nel..."
1,geo soilan park - battery park city,battery park city,other,av,M-01,-74.016890,40.712719,M,battery park city,100,battery park city,M,[M283],"[nelson rockefeller park playground north, nel..."
2,brookville park,brookville park,park_direct,av,Q-13,-73.745642,40.659540,Q,brookville park,100,brookville park,Q,[Q008],[brookville park]
3,highland park,highland park,park_direct,av,B-05,-73.886671,40.684980,B,highland park,100,highland park,Q,[Q020],"[highland park, upper highland playground, low..."
4,highland park,highland park,park_direct,av,Q-05,-73.888353,40.687724,Q,highland park,100,highland park,Q,[Q020],"[highland park, upper highland playground, low..."


In [1]:
# DPR.head()
# DPR[DPR.ParkID.str.len() >1 ]

#### now for those where there are multiple ParkID we need to randomly pick one or another for each call

## Matching calls

In [13]:
calls = pd.read_csv(PARQA + 'data/311/311DPR.csv', encoding='utf8', na_values='Unspecified')
calls = calls[calls.Descriptor=='Garbage or Litter']

In [14]:
# calls.Descriptor.value_counts()

In [15]:
myCalls = calls[['Park Facility Name','Descriptor','Created Date','Closed Date','Longitude','Latitude','Location Type', 'Complaint Type']]
myCalls['Park Facility Name'] = myCalls['Park Facility Name'].str.lower()
myCalls['Park Facility Name'].head()

notNamedCalls = myCalls[pd.isnull(myCalls['Park Facility Name'])]
namedCalls = myCalls[pd.notnull(myCalls['Park Facility Name'])]

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [16]:
ncMatched = namedCalls.merge(DPR, how='left',left_on='Park Facility Name', right_on='cleanName')
ncMatched.head(1)

NameError: name 'DPR' is not defined

In [307]:
DPR[DPR.cleanName =='highland park']

,cleanName,bestMatch,ParkID
3,highland park,highland park,NaN
4,highland park,highland park,[Q020]
5,highland park,highland park,NaN
6,highland park,highland park,[Q020]


In [306]:
# ncMatched['sParkID'] = nc.Matched
ncMatched[pd.isnull(ncMatched['ParkID'])].head()


,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type,Complaint Type,cleanName,bestMatch,ParkID
3,highland park,Snow or Ice,12/31/2010 02:57:34 PM,01/03/2011 11:31:26 AM,NaN,NaN,Park,Maintenance or Facility,highland park,highland park,NaN
5,highland park,Snow or Ice,12/31/2010 02:57:34 PM,01/03/2011 11:31:26 AM,NaN,NaN,Park,Maintenance or Facility,highland park,highland park,NaN
13,col david marcus playground,Snow or Ice,12/31/2010 09:12:19 AM,01/03/2011 12:16:41 PM,NaN,NaN,Park,Maintenance or Facility,NaN,NaN,NaN
17,mccarren park - vincent v abate playground,Snow or Ice,12/30/2010 10:12:25 PM,01/03/2011 02:28:11 PM,NaN,NaN,Park,Maintenance or Facility,NaN,NaN,NaN
20,forest park - giovannelli playground,Dog Off Leash,12/30/2010 09:31:36 PM,12/30/2010 09:59:44 PM,NaN,NaN,Park,Animal in a Park,NaN,NaN,NaN


## HERE I NEED TO SPATIALLY JOIN GEOLOCATED ONES